# English to Chinese  translator

In [20]:
# import libraries
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [2]:
# load data
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 1000
import pandas as pd
data_path =r'C:\Users\amuez\Downloads\cmn-eng/cmn.txt'

In [3]:
# Vectorization of data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding = 'utf-8') as f:
    lines = f.read().split('\n')
for line in lines [: min(num_samples, len(lines) -1)]:
    input_text, target_text, _ = line.split('\t')
    
    target_text = '\t' + target_text +'\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [4]:
#input_characters
len(target_characters)

802

In [5]:
#input and target characters
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [6]:
#Samples and Length

print('Number of samples:', len(input_texts))
print('Number of unique input token:', num_encoder_tokens)
print('Number of unique output tokens:',num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 1000
Number of unique input token: 62
Number of unique output tokens: 802
Max sequence length for inputs: 15
Max sequence length for outputs: 13


In [7]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [8]:
input_token_index, target_token_index

({' ': 0,
  '!': 1,
  "'": 2,
  ',': 3,
  '.': 4,
  '0': 5,
  '1': 6,
  '3': 7,
  '7': 8,
  ':': 9,
  '?': 10,
  'A': 11,
  'B': 12,
  'C': 13,
  'D': 14,
  'E': 15,
  'F': 16,
  'G': 17,
  'H': 18,
  'I': 19,
  'J': 20,
  'K': 21,
  'L': 22,
  'M': 23,
  'N': 24,
  'O': 25,
  'P': 26,
  'Q': 27,
  'R': 28,
  'S': 29,
  'T': 30,
  'U': 31,
  'V': 32,
  'W': 33,
  'Y': 34,
  'Z': 35,
  'a': 36,
  'b': 37,
  'c': 38,
  'd': 39,
  'e': 40,
  'f': 41,
  'g': 42,
  'h': 43,
  'i': 44,
  'j': 45,
  'k': 46,
  'l': 47,
  'm': 48,
  'n': 49,
  'o': 50,
  'p': 51,
  'q': 52,
  'r': 53,
  's': 54,
  't': 55,
  'u': 56,
  'v': 57,
  'w': 58,
  'x': 59,
  'y': 60,
  'z': 61},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '.': 4,
  '1': 5,
  '3': 6,
  '7': 7,
  '?': 8,
  'D': 9,
  'J': 10,
  'T': 11,
  'e': 12,
  'i': 13,
  'm': 14,
  'o': 15,
  'r': 16,
  't': 17,
  'w': 18,
  '。': 19,
  '一': 20,
  '万': 21,
  '三': 22,
  '上': 23,
  '下': 24,
  '不': 25,
  '世': 26,
  '东': 27,
  '丟': 28,
  '丢': 29,
  '严'

In [9]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype = 'float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype = 'float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype = 'float32')

In [10]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t-1, target_token_index[char]] = 1.
    decoder_input_data[i, t+1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [11]:
encoder_input_data[0].shape

(15, 62)

In [12]:
# Encoder
# input sequence
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]


In [13]:
#decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences = True, return_state = True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                    initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# LSTM Model and Run Training

In [14]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#Training the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
             metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
         batch_size=batch_size,
         epochs=epochs,
         validation_split=0.2)

Epoch 1/100
13/13 [==============================] - 9s 433ms/step - loss: 4.7898 - accuracy: 0.4127 - val_loss: 2.6223 - val_accuracy: 0.5342
Epoch 2/100
13/13 [==============================] - 2s 182ms/step - loss: 2.3173 - accuracy: 0.5782 - val_loss: 2.6341 - val_accuracy: 0.5231
Epoch 3/100
13/13 [==============================] - 3s 197ms/step - loss: 2.2499 - accuracy: 0.5726 - val_loss: 2.7544 - val_accuracy: 0.5200
Epoch 4/100
13/13 [==============================] - 2s 193ms/step - loss: 2.1673 - accuracy: 0.5875 - val_loss: 2.7055 - val_accuracy: 0.5188
Epoch 5/100
13/13 [==============================] - 2s 193ms/step - loss: 2.1354 - accuracy: 0.5891 - val_loss: 2.5623 - val_accuracy: 0.5538
Epoch 6/100
13/13 [==============================] - 2s 192ms/step - loss: 2.0783 - accuracy: 0.6091 - val_loss: 2.5708 - val_accuracy: 0.5600
Epoch 7/100
13/13 [==============================] - 2s 180ms/step - loss: 2.0535 - accuracy: 0.6142 - val_loss: 2.6079 - val_accuracy: 0.5496

In [15]:
model.save('chinese_to_eng.h5')

In [16]:
#sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state= decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())





In [17]:
def decode_sequence(input_seq):
    # Encoding the input as state vector
    states_value = encoder_model.predict(input_seq)
    # Generating empty target sequence of length 1.
    target_seq = np.zeros((1,1, num_decoder_tokens))
    target_seq[0,0, target_token_index['\t']] = 1.
    
    # to sample loop for a batch of sequences
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens,h,c = decoder_model.predict(
            [target_seq] + states_value)
        
        # to sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        
        decoded_sentence += sampled_char
        
        if (sampled_char =='\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
         # to update the target sequence
        target_seq = np.zeros((1,1, num_decoder_tokens))
        target_seq[0,0, sampled_token_index] = 1.
        # to update status
        states_value = [h,c]
        
    return decoded_sentence


In [18]:
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    # to decode the sentence
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: 你好。

-
Input sentence: Hi.
Decoded sentence: 你好。

-
Input sentence: Run.
Decoded sentence: 你用跑的。

-
Input sentence: Wait!
Decoded sentence: 等等！

-
Input sentence: Wait!
Decoded sentence: 等等！

-
Input sentence: Begin.
Decoded sentence: 开始！

-
Input sentence: Hello!
Decoded sentence: 你好。

-
Input sentence: I won!
Decoded sentence: 我赢了。

-
Input sentence: Oh no!
Decoded sentence: 不会吧。

-
Input sentence: Cheers!
Decoded sentence: 乾杯!

-
Input sentence: Got it?
Decoded sentence: 你懂了吗？

-
Input sentence: He ran.
Decoded sentence: 他跑了。

-
Input sentence: Hop in.
Decoded sentence: 跳进来。

-
Input sentence: I quit.
Decoded sentence: 我不能走。

-
Input sentence: I quit.
Decoded sentence: 我不能走。

-
Input sentence: I'm OK.
Decoded sentence: 我是個英雄。

-
Input sentence: I'm up.
Decoded sentence: 我是个男人。

-
Input sentence: Listen.
Decoded sentence: 听着。

-
Input sentence: No way!
Decoded sentence: 没门！

-
Input sentence: No way!
Decoded sentence: 没门！

-
Input sentence: Rea

Note: you may need to restart the kernel to use updated packages.
